In [13]:
%matplotlib inline
import re
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import Counter
from cleantext import clean
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from collections import Counter
import subprocess
import zipfile
import io
import csv
import zipfile38 as zipfile
from dask import dataframe as dd
from sklearn.model_selection import train_test_split
import h5py
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/simonmork/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Define settings for cleaning
def clean_text(text):
    """Calls the clean function from cleantext clean on a string"""
    t = clean(text,
    fix_unicode=False,               # fix various unicode errors
    to_ascii=True,                  # transliterate to closest ASCII representation
    lower=True,                     # lowercase text
    no_line_breaks=True,         # fully strip line breaks as opposed to only normalizing them NOT WORKING?
    no_urls=True,                  # replace all URLs with a special token
    no_emails=True,                # replace all email addresses with a special token
    no_phone_numbers=False,         # replace all phone numbers with a special token
    no_numbers=True,               # replace all numbers with a special token
    no_digits=False,                # replace all digits with a special token
    no_currency_symbols=False,      # replace all currency symbols with a special token
    no_punct=True,                 # remove punctuations
    replace_with_punct="",          # instead of removing punctuations you may replace them
    replace_with_url="URL",
    replace_with_email="EMAIL",
    replace_with_phone_number="PHONE",
    replace_with_number="NUM",
    replace_with_digit="0",
    replace_with_currency_symbol="CUR",
    lang="en"                       # set to 'de' for German special handling
    )
    return t


pd.set_option("display.max_colwidth", 10000000)    
print("Done")    
    


Done


In [15]:
#Small df
print("Getting csv...")
df = pd.read_csv('https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv')
print(type(df))

Getting csv...
<class 'pandas.core.frame.DataFrame'>


In [114]:
csv_textfilereader = pd.read_csv('/Users/simonmork/Desktop/GitHub/FinalProjectDS/news_cleaned_2018_02_13.csv', chunksize=1000000, iterator=True)

first_chunk = csv_textfilereader.get_chunk(2)
first_article = first_chunk.iloc[1]
print("len: ", len(first_chunk))
print( first_article)

len:  2
Unnamed: 0                                                                                                                                                                                                                                                                                                                                                                         1
id                                                                                                                                                                                                                                                                                                                                                                                 6
domain                                                                                                                                                                                                                                                

In [98]:
first_article = first_chunk.iloc[4]
print(first_article)

Unnamed: 0                                                                                                                                                                                                                            4
id                                                                                                                                                                                                                                    9
domain                                                                                                                                                                                                               barenakedislam.com
type                                                                                                                                                                                                                               hate
url                                                                     

In [77]:
# Part 1 - Task 3
#Big df
csv_textfilereader = pd.read_csv('/Users/simonmork/Desktop/GitHub/FinalProjectDS/news_cleaned_2018_02_13.csv', chunksize=1000000)


print(csv_textfilereader.get_chunk(4))


#for i, current_df in enumerate(df):
    #print(current_df)
    #print(i)



   Unnamed: 0  id              domain   type  \
0           0   2       express.co.uk  rumor   
1           1   6  barenakedislam.com   hate   
2           2   7  barenakedislam.com   hate   
3           3   8  barenakedislam.com   hate   

                                                                                                                                                                                                                url  \
0                                                                                                https://www.express.co.uk/news/science/738402/life-an-ILLUSION-reality-does-not-exist-if-you-are-not-looking-at-it   
1                                                                                                                                                          http://barenakedislam.com/category/donald-trump/page/43/   
2                                                                                                                 

In [43]:
    # Clean dataframe function
def clean_df(df):
        print("Getting headers..")
        column_names = list(df.columns.values)
        #print(column_names)

        print("Cleaning...")
        for index, row in df.iterrows():
            #print(df.at[index,'content'])
            cleaned = clean_text(row['content'])
            df.at[index,'content'] = cleaned
            #print(cleaned)
            
            
        # Split the content column into a list of words
        words = df['content'].str.split()
        # Create a set of unique words
        unique_words = set(word for word_list in words for word in word_list)
        # Count the number of unique words
        num_unique_words_after_clean = len(unique_words)
        print(num_unique_words_after_clean) 



        print("Tokenizing and removing stopwords...")
        stop_words = set(stopwords.words('english'))
        for index, row in df.iterrows():    
            #print(df.at[index,'content'])
            df.at[index,'content'] = nltk.word_tokenize(row['content'])
            
            filtered_sentence = []
            for w in df.at[index,'content']:
                if w not in stop_words:
                    filtered_sentence.append(w)
            
            df.at[index,'content'] = filtered_sentence
            #print(filtered_sentence)
        

        # concatenate all the lists into a single list of words
        words = [word for content in df['content'] for word in content]
        # get the set of unique words
        unique_words = set(words)
        # get the number of unique words
        num_unique_words_after_stop = len(unique_words)
        print(num_unique_words_after_stop)
        print(1-num_unique_words_after_stop/num_unique_words_after_clean)
            
            

        print("Stemming")
        snowball = SnowballStemmer(language='english')
        for index, row in df.iterrows():   
            stemmed = []
            for word in df.at[index,'content']:
                stemmed.append(snowball.stem(word))
            df.at[index,'content'] = stemmed
            #print(stemmed)


        # concatenate all the lists into a single list of words
        words = [word for content in df['content'] for word in content]
        # get the set of unique words
        unique_words = set(words)
        # get the number of unique words
        num_unique_words_after_stem = len(unique_words)
        print(num_unique_words_after_stem)
        print(1-num_unique_words_after_stem/num_unique_words_after_stop)


        print("Done...")

In [36]:
#do it for first dataframe
var = csv_textfilereader.get_chunk(1)
print(var.content)
clean_df(var)
print(var['content'])
df = var

12    "…I have set before you life and death, blessing and cursing’\n\ntherefore choose life, that both you and your descendants may live."\n\n--Deuteronomy 30:19 (NKJV)\n\nOn the crisp, sunny, fall Columbus Day in 1999, organizers of the "Say So" march approached the steps of the U.S. Supreme Court. The marchers, who were predominantly black pastors and lay persons, concluded their three-day protest at the site of two monumental cases: the school desegregation Brown v. Board of Education (1954) and the pro-abortion Roe v. Wade "rights" in t he latterconverged in the declaration of Rev. Johnny M. Hunter, the march’s sponsor and national director of Life, Education and Resource Network (LEARN), the largest black pro-life organization.\n\n‘"Civil rights’ doesn’t mean anything without a right to life!" declared Hunter. He and the other marchers were protesting the disproportionately high number of abortions in the black community. The high number is no accident. Many Americansblack and 

In [9]:
#Part 1 - Task 2
print("Counting number of urls, dates and num, and fake news and trump")
url = 0
num = 0
trymp=0
fakenews = 0
notfake = 0
for index, row in df.iterrows():
    #print(data['content'])
    for word in df.at[index,'content']:
        #print(word)
        if word == 'url':
            url = url+1
        if word == 'num':
            num = num+1
        if word == 'trump':
            trymp = trymp+1
    if df.at[index,'type'] == 'fake':
        fakenews = fakenews+1
    else:
        notfake = notfake+1
print("Num urls: ", url)
print("Num num: ", num)
print("Num trymp: ", trymp)
print("Num fake: ", fakenews)
print("Num notfake: ", notfake)

Counting number of urls, dates and num, and fake news and trump
Num urls:  0
Num num:  0
Num trymp:  0
Num fake:  155
Num notfake:  95


In [19]:
# Part1 - Task 4

X, y = df.content, df['type']

#print(X)
#print("y: ", y)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size = 0.2, random_state = 0)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 0)

print("train: ")
print(X_train, y_train)


print("test: ")
print(X_test, y_test)

print("Validation: ")
print(X_val, y_val)



ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [ ]:
def make_bin(to_bin):
    to_bin = np.asarray(to_bin)
    reliable = 0
    fake = 0
    bin_list = []
    for i in range(len(to_bin)):
        if to_bin[i] != 'fake':
            bin_list.append(0)
            reliable = reliable +1
        else:
            bin_list.append(1)
            fake = fake+1
    print(reliable)
    print(fake)
    return bin_list

In [ ]:
y_train_bin = make_bin(y_train)
print("y_train: ", y_train_bin)
y_val_bin = make_bin(y_val)
print("y_val: ", y_val_bin)
y_test_bin = make_bin(y_test)
print("y_test: ", y_test_bin)

82
118
y_train:  [0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1]
10
15
y_val:  [0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1]
3
22
y_test:  [1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]


In [ ]:
def X_vectoriser(vectee):
    vectee = [' '.join(words) for words in vectee]
    vectorizer = CountVectorizer(vocabulary=unique_words)
    vectee = vectorizer.fit_transform(vectee)
    return vectee

In [ ]:
X_train_vec = X_vectoriser(X_train)
print("X_train_vec: ", X_train_vec)
X_val_vec = X_vectoriser(X_val)
print("X_val_vec: ", X_val_vec)
X_test_vec = X_vectoriser(X_test)
print("X_test_vec: ", X_test_vec)

X_train_vec:    (0, 73)	4
  (0, 91)	1
  (0, 112)	1
  (0, 119)	1
  (0, 127)	1
  (0, 160)	2
  (0, 164)	1
  (0, 178)	1
  (0, 199)	1
  (0, 207)	1
  (0, 223)	1
  (0, 255)	1
  (0, 267)	2
  (0, 288)	1
  (0, 304)	1
  (0, 321)	1
  (0, 364)	2
  (0, 375)	4
  (0, 415)	1
  (0, 440)	1
  (0, 462)	1
  (0, 473)	1
  (0, 481)	2
  (0, 493)	1
  (0, 500)	1
  :	:
  (198, 10354)	1
  (198, 10386)	1
  (198, 10399)	2
  (198, 10469)	1
  (198, 10486)	1
  (198, 10547)	1
  (198, 10563)	3
  (198, 10601)	1
  (198, 10633)	2
  (198, 10650)	1
  (198, 10653)	1
  (198, 10733)	1
  (198, 10794)	1
  (198, 10888)	5
  (198, 10932)	1
  (198, 10944)	1
  (198, 10971)	1
  (199, 1659)	1
  (199, 2357)	2
  (199, 2612)	1
  (199, 2933)	1
  (199, 3083)	1
  (199, 5604)	1
  (199, 9807)	1
  (199, 10177)	1
X_val_vec:    (0, 368)	1
  (0, 603)	1
  (0, 952)	2
  (0, 1462)	1
  (0, 1529)	1
  (0, 1589)	1
  (0, 1632)	1
  (0, 1968)	1
  (0, 2033)	1
  (0, 2153)	1
  (0, 2169)	1
  (0, 2332)	1
  (0, 2337)	1
  (0, 2349)	2
  (0, 2678)	2
  (0, 2768)	1
  (0, 

In [7]:
simple_log_model = LogisticRegression(max_iter=100000)
simple_log_model.fit(X_train_vec, y_train_bin)
y_pred = simple_log_model.predict(X_val_vec)

print(accuracy_score(y_val_bin, y_pred))

NameError: name 'X_train_vec' is not defined

In [ ]:
#tips: nearest neighbor on missing data